In [2]:
"""Template for jupyter notebooks created within this directory. Adds the parent directory to path and sets autoreload."""

'Template for jupyter notebooks created within this directory. Adds the parent directory to path and sets autoreload.'

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
import os
import sys

# Get the current working directory
cwd = os.getcwd()

# Get the parent directory
parent_dir = os.path.dirname(cwd)

# Get the grandparent directory (two levels above)
root_dir = os.path.dirname(parent_dir)

# Add the root directory to sys.path
if root_dir not in sys.path:
    sys.path.append(root_dir)

print(f"Root directory: {root_dir} is added to sys.path")


Root directory: /Users/aag/Documents/proteinfolding is added to sys.path


In [3]:
## test imports

from proteinfolding import * ## should run without errors

##TODO: proper testing ##

In [6]:
from proteinfolding.supporting_functions import get_qubo_hamiltonian
num_res = 4
num_rot = 3

Q = get_qubo_hamiltonian(num_rot, num_res)

/Users/aag/Documents/proteinfolding/notebooks/local_testing
┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│         See LICENSE.PyRosetta.md or email license@uw.edu for details         │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2025 [Rosetta PyRosetta4.Release.python310.m1 2025.06+release.029c6a159b896477003a14f78f472d4cd2cead46 2025-02-04T15:14:13] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.Release.pyt

In [11]:
from docplex.mp.model import Model
import numpy as np

n = Q.shape[0]
mdl = Model(name='QUBO_ILP')

# Create binary variables: x[i][j] for residue i and rotamer j
x = [[mdl.binary_var(name=f"x_{i}_{j}") for j in range(num_rot)] for i in range(num_res)]

# Add one-hot (Hamming weight = 1) constraint per residue
for i in range(num_res):
    mdl.add_constraint(mdl.sum(x[i][j] for j in range(num_rot)) == 1,
                       ctname=f"one_hot_residue_{i}")
    
x_flat = [var for row in x for var in row]  # x[res][rot] → x_flat[index]

# Auxiliary variables for products x_i * x_j
y = {}
for i in range(n):
    for j in range(i+1, n):
        y[i, j] = mdl.binary_var(name=f'y_{i}_{j}')
        mdl.add_constraint(y[i, j] <= x_flat[i])
        mdl.add_constraint(y[i, j] <= x_flat[j])
        mdl.add_constraint(y[i, j] >= x_flat[i] + x_flat[j] - 1)

# Build the objective
objective = 0
for i in range(n):
    objective += Q[i, i] * x_flat[i]
    for j in range(i+1, n):
        objective += Q[i, j] * y[i, j]

mdl.minimize(objective)
mdl.solve()

# Output
solution = [[v.solution_value for v in row] for row in x]  # returns a 2D list: [residue][rotamer]
print("Optimal solution:", solution)
print("Objective value:", mdl.objective_value)


Optimal solution: [[0, 1.0, 0], [0, 1.0, 0], [1.0, 0, 0], [0, 1.0, 0]]
Objective value: -0.8408074975013732
